# Evaluation of the number and amount of anchor text pointing to target documents

- Für jeden der 7 Snapshots schauen wie viele Anchortexte im Average drauf zeigen?
- For each of the 7 anchor text snapshots: how many anchor texts point the the target documents?
  - How many of them have anchor text
  - How many anchor text on average

In [1]:
from trectools import TrecQrel
import pandas as pd

QREL_DIR = '/mnt/ceph/storage/data-tmp/2021/kibi9872/thesis-probst/Data/navigational-topics-and-qrels-ms-marco-v1/'
DATA_DIR = '/mnt/ceph/storage/data-tmp/2021/kibi9872/ms-marco-v1-anchor-text-concatenated/ms-marco-v1/anchor-text/'
ALL_SNAPSHOTS = ['2016-07', '2017-04', '2018-13', '2019-47', '2020-05', '2021-04', 'union-2016-to-2021']

random_entry_page_docs = TrecQrel(QREL_DIR + 'qrels.msmarco-entrypage-random.txt').qrels_data.docid.unique()
popular_entry_page_docs = TrecQrel(QREL_DIR + 'qrels.msmarco-entrypage-popular.txt').qrels_data.docid.unique()
all_docs = list(random_entry_page_docs) + list(popular_entry_page_docs)

In [2]:
def aggregate_dataset(snapshot_name):
    import gzip
    import json
    from tqdm import tqdm
    ret = {}
    
    with gzip.open(DATA_DIR + 'common-crawl-' + snapshot_name + '-to-ms-marco-v1-no-metadata.jsonl.gz', 'r') as f:
        for l in tqdm(f):
            l = json.loads(l)
            if l['id'] in all_docs:
                ret[l['id']] = len(l['anchors'])

    return ret

def create_df_for_qrels(docs):
    df = []
    for snapshot in ALL_SNAPSHOTS:
        empty_docs = 0
        sum_anchors = 0
        for doc in docs:
            if doc in ALL_ANCHORS[snapshot]:
                sum_anchors += ALL_ANCHORS[snapshot][doc]
            else:
                empty_docs += 1
        df += [{'Snapshot': snapshot, 'average-anchors-per-target-document': sum_anchors/len(docs), 'target-documents-without-incoming-anchors': empty_docs}]

    return pd.DataFrame(df)

ALL_ANCHORS = {i: aggregate_dataset(i) for i in ALL_SNAPSHOTS}

830830it [00:14, 58450.43it/s]
1181185it [00:18, 64843.83it/s]
1266901it [00:22, 57175.97it/s]
1163273it [00:19, 60152.99it/s]
1235843it [00:22, 55243.89it/s]
1146746it [00:18, 61566.41it/s]
1703834it [00:49, 34698.33it/s]


# Random Entry Pages

In [3]:
create_df_for_qrels(random_entry_page_docs)

,Snapshot,average-anchors-per-target-document,target-documents-without-incoming-anchors
0,2016-07,137.252525,14
1,2017-04,170.080808,7
2,2018-13,234.828283,5
3,2019-47,196.171717,6
4,2020-05,212.515152,4
5,2021-04,193.565657,7
6,union-2016-to-2021,451.848485,1


# Popular Entry Pages

In [4]:
create_df_for_qrels(popular_entry_page_docs)

,Snapshot,average-anchors-per-target-document,target-documents-without-incoming-anchors
0,2016-07,934.858586,1
1,2017-04,924.737374,0
2,2018-13,934.626263,0
3,2019-47,910.919192,0
4,2020-05,915.020202,0
5,2021-04,907.666667,0
6,union-2016-to-2021,977.838384,0
